# 滑动窗口数据采样

In [1]:
from importlib.metadata import version
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [8]:
with open("/Users/tangren/Documents/Build-a-Large-Language-Model-From-Scratch-/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

enc_sample = enc_text[50:]

5145


In [9]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x:{x}")
print(f"y:{y}")

x:[290, 4920, 2241, 287]
y:[4920, 2241, 287, 257]


In [11]:
for i in range (1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [12]:
import torch 
from torch.utils.data import Dataset, DataLoader

In [ ]:
# 滑动窗口数据采样
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

            


In [16]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True,num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    return dataloader


In [17]:
with open("/Users/tangren/Documents/Build-a-Large-Language-Model-From-Scratch-/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)

print(first_batch)






[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [19]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


In [20]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print(targets)





tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 创建词嵌入

In [31]:
input_ids = torch.tensor([[2,3,5,1]])

vocab_size = 6
output_dim = 6

In [32]:
torch.manual_seed(123)
embedding = torch.nn.Embedding(vocab_size, output_dim)

print(embedding.weight)




Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603],
        [-0.2196, -0.3792,  0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205],
        [-0.1690,  0.9178, -0.3885, -0.9343, -0.4991, -1.0867],
        [ 0.9624,  0.2492, -0.4845, -2.0929,  0.0983, -0.0935],
        [ 0.2662, -0.5850, -0.3430, -0.6821, -0.9887, -1.7018]],
       requires_grad=True)


In [33]:
print(embedding(input_ids))

tensor([[[ 0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205],
         [-0.1690,  0.9178, -0.3885, -0.9343, -0.4991, -1.0867],
         [ 0.2662, -0.5850, -0.3430, -0.6821, -0.9887, -1.7018],
         [-0.2196, -0.3792,  0.7671, -1.1925,  0.6984, -1.4097]]],
       grad_fn=<EmbeddingBackward0>)


In [37]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs: ", inputs)
print("Inputs shape: ", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print("Token embeddings shape: ", token_embeddings.shape)




Token IDs:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:  torch.Size([8, 4])
Token embeddings shape:  torch.Size([8, 4, 256])


In [38]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_emberddings = pos_embedding_layer(torch.arange(context_length))
print(pos_emberddings.shape)






torch.Size([4, 256])


In [39]:
input_embeddings = token_embeddings + pos_emberddings #词向量和位置向量相加
print(input_embeddings.shape)

torch.Size([8, 4, 256])
